# 零件白斑缺陷检测系统 - 训练



## 1. 环境准备

首先导入必要的库：

In [ ]:
from ultralytics import YOLO
import yaml
from pathlib import Path
import os
import matplotlib.pyplot as plt
import torch
import sys

## 2. 设置工作目录

In [ ]:
# 设置项目根目录
project_root = Path("D:\\Program\\defect_detection")
os.chdir(project_root)
print(f"当前工作目录: {os.getcwd()}")

## 3. 检查GPU可用性

In [ ]:
print(f"CUDA是否可用: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU型号: {torch.cuda.get_device_name(0)}")
    print(f"可用GPU数量: {torch.cuda.device_count()}")

## 4. 加载配置文件

In [ ]:
# 加载训练配置
config_path = project_root / 'configs' / 'config.yaml'
with open(config_path, 'r', encoding='utf-8') as f:
    cfg = yaml.safe_load(f)

print("训练配置信息:")
print(f"epochs: {cfg['train']['epochs']}")
print(f"batch_size: {cfg['train']['batch_size']}")
print(f"image_size: {cfg['train']['img_size']}")

## 5. 数据集检查

In [ ]:
# 检查数据集配置
data_yaml_path = project_root / 'configs' / 'data.yaml'
with open(data_yaml_path, 'r', encoding='utf-8') as f:
    data_cfg = yaml.safe_load(f)

print("数据集配置信息:")
print(f"训练集路径: {data_cfg.get('train', '未指定')}")
print(f"验证集路径: {data_cfg.get('val', '未指定')}")
print(f"类别数量: {len(data_cfg.get('names', []))}")
print(f"类别名称: {data_cfg.get('names', [])}")

## 6. 选择预训练模型

可选模型：
- yolov5su.pt: 标准版（推荐）
- yolov5mu.pt: 中等版本
- yolo11n.pt: 轻量版本

In [ ]:
# 选择预训练模型
model_path = project_root / 'yolov5su.pt'
model = YOLO(str(model_path))
print(f"已加载模型: {model}")

## 7. 开始训练

In [ ]:
# 开始训练
results = model.train(
    data=str(data_yaml_path),
    epochs=cfg['train']['epochs'],
    batch=cfg['train']['batch_size'],
    imgsz=cfg['train']['img_size'],
    device=0 if torch.cuda.is_available() else 'cpu',
    pretrained=True,
    resume=False,
    plots=True,  # 生成训练过程图表
    save=True,   # 保存训练结果
    save_period=10  # 每10个epoch保存一次
)

## 8. 查看训练结果

In [ ]:
# 显示训练结果
print("\n训练完成！")
print(f"最佳模型保存在: {model.best}")

# 如果有results对象，显示训练指标
if results is not None:
    print("\n训练指标:")
    print(f"最佳mAP: {results.maps}")
    print(f"最终损失: {results.results_dict['train/box_loss'][-1]}")
    
    # 显示训练曲线
    results_file = Path('runs/train/results.csv')
    if results_file.exists():
        import pandas as pd
        results_df = pd.read_csv(results_file)
        
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(results_df['epoch'], results_df['train/box_loss'], label='训练损失')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('训练损失曲线')
        plt.legend()
        
        plt.subplot(1, 2, 2)
        plt.plot(results_df['epoch'], results_df['metrics/mAP50(B)'], label='mAP@0.5')
        plt.xlabel('Epoch')
        plt.ylabel('mAP')
        plt.title('mAP曲线')
        plt.legend()
        
        plt.tight_layout()
        plt.show()

## 9. 验证模型

In [ ]:
# 加载训练好的最佳模型进行验证
trained_model = YOLO(model.best)
validation_results = trained_model.val(
    data=str(data_yaml_path),
    split='val'
)

print("\n验证结果:")
print(f"验证集mAP@0.5: {validation_results.box.map50}")
print(f"验证集mAP@0.5:0.95: {validation_results.box.map}")

## 10. 模型导出（可选）

In [ ]:
# 导出模型为ONNX格式（可选）
export_path = project_root / 'models' / 'best_model.onnx'
trained_model.export(format='onnx', imgsz=cfg['train']['img_size'])
print(f"模型已导出到: {export_path}")